# A Research about Named Entity Extraction 

## Introduction
In Diego Marcheggiani and Ivan Titov’s paper Encoding Sentences with Graph Convolutional Networks for Semantic Role Labeling, researchers found that the results of syntactic analysis and the results of semantic role labeling are mostly mirrored. Based on the above observations, researchers proposed a graph-based convolutional network(GCN) method for semantic role labeling. Use GCN to encode a syntax-dependent tree, resulting in a potential feature representation of the word in the sentence. The article observes that the GCN layer is complementary with the LSTM layer: when the GCN layer and the LSTM layer are superimposed, their performance gets the latest state-of-the-art.

In order to test the behavior of the GCN model, we implemented a complete GCN model using python and tensorflow. If fact, in the open source project provided by the model, it gives a demo to user that help them to use the GCN with their own data. Follow the instructions, our research group use CoNLL 2003. as our dataet and try to use GCN to do Named Entity Extraction here.

To feed the GCN model with our own data, we need to do preprocess on our raw dataset, CoNLL 2003 dataset contains thousands of sentences from newspaper and magazine. <br>
But GCN model need three matrices:<br>
First one is a N\*N adjacency matrix that represent the relationship between each nodes, here N represent the number of nodes in a graph. <br>
The second one is a N\*D matrix that represent the feature vector of each node, here D is the number of features of a node. <br>
The third matrix is an N\*E matrix that reprensent the real class of each node, here E is the number of classes.

But in our project the nodes in a graph is actually words in a sentence. So we use Stanford Parser to help us to get the relationship in a sentence. With the dependencies provided by Stanford Parser, the N*N adjacency matrix can be build easily.



Another tool we use in this project is word2vec, word2vec is a two-layer neural net that processes text. Its input is a text corpus and its output is a set of vectors: feature vectors for words in that corpus. With the help of this tool, for each word in a sentence, we can get a unique 300 dimensions vector to represent it, we use this vector as this word’s feature vector and build the N*D matrix. In this project, we set the length of feature vector 300 so here the value of D is 300.

For the N*E matrix, the CoNLL 2003 gives the exact named entity of each word. Our research group use python to extract them and build them into matrix. 

### Here is how we preprocess data in order to feed into GCN model


#### The preprocess of GCN Entity Extraction project.
- Provide functions that generate three matrix for GCN's augments
- Improve functions, for sentences, dependencies, vectors and classes,
- Use a dictionary to store them, only need to set the index of sentence.
- Run this program to store four  dictionary in the local memory.
- Extract needed matrix using find functions, or use operate[]



In [1]:
import time
from numpy import *
import numpy as np
import re
import scipy.sparse as sp


#### get all sentences from a file. set the first element as key and others are set as value

In [2]:
def getSentences(dataset_path):
    sentences = {}
    indexes = []
    for line in open(dataset_path):
        line = line.split()
        index = line[0]
        sentence = line[1:]
        sentences[index] = sentence
        indexes.append(index)
    return sentences, indexes

#### select sentences, return sentences that word number is greater than number

In [3]:
def select(sentences, number):
    print("Reading the sentences ...\n")
    sens = {}
    for sen in sentences.items():
        if (len(sen[1]) >= number):
            sens[sen[0]] = sen[1]
    return sens

#### get dependencies from a file, set the index as key and dependency matrix as value, get the dependency matrix by the given index

In [4]:
def getDependencies(dependency_path, sentences):
    print("Reading the dependencies...\n")
    deps = {}
    temp = []
    for line in open(dependency_path):
        line = line.split()
        temp.append(line)
    for i in range(len(temp)):
        if(temp[i] != [] and temp[i][0] == "Index"):
            index = temp[i][1]
            length = len(sentences[index])
            matrix = zeros([length, length], dtype=int8)
            for dep in temp[i+1]:
                dep = re.split('-|,|\(|\)', dep)
                x = int(dep[2]) - 1
                y = int(dep[4]) - 1
                matrix[x][y] = 1
            deps[index] = matrix
    return deps

In [5]:
def findMatrix(index, dependencies):
    return dependencies[index]

#### get word vectors from a file, use index as key and dependency matrix as value

In [6]:
def getVectors(wordvector_path, sentences):
    print("Reading the vectors...\n")
    vecs = {}
    temp = []
    for line in open(wordvector_path):
        line = line.split()
        temp.append(line)
    for i in range(len(temp)):
        print(i)
        if(temp[i][0] == "Index"):
            index = temp[i][1]
            if(index not in sentences):
                continue
            length = len(sentences[index])
            matrix = zeros([length, 300], dtype=float)
            for j in range(length):
                vector = temp[i+j+1]
                if(vector[1] == "N/A"):
                    continue
                else:
                    for k in range(300):
                        matrix[j][k] = float(vector[k+1])
            vecs[index] = matrix
    return vecs


In [7]:
def findVectors(index, vectors):
    return vectors[index]

#### get word label from a file, use index as key and label matrix as value

In [8]:
def getClass(wordclass_path, sentences):
    print("Reading the classes...\n")
    classes = {}
    temp = []
    for line in open(wordclass_path):
        line = line.replace("<", "|<")
        line = line.replace(">", "> ")
        line = re.split("\|", line)
        index = line[0]
        line = line[1:]
        if (index not in sentences):
            continue
        matrix = zeros([5, len(sentences[index])], dtype=int8)
        for item in line:
            item = item.split()
            sentence = np.array(sentences[index])
            if(item[0] == "<O>"):
                objects = item[1:]
                for obj in objects:
                    ii = np.where(sentence == obj)[0]
                    for i in ii:
                        matrix[0][i] = 1
            if(item[0] == "<ORG>"):
                orgs = item[1:]
                for org in orgs:
                    ii = np.where(sentence == org)[0]
                    for i in ii:
                        matrix[1][i] = 1
            if(item[0] == "<PER>"):
                persons = item[1:]
                for person in persons:
                    ii = np.where(sentence == person)[0]
                    for i in ii:
                        matrix[2][i] = 1
            if (item[0] == "<MISC>"):
                maliciouses = item[1:]
                for malicious in maliciouses:
                    ii = np.where(sentence == malicious)[0]
                    for i in ii:
                        matrix[3][i] = 1
            if (item[0] == "<LOC>"):
                locations = item[1:]
                for location in locations:
                    ii = np.where(sentence == location)[0]
                    for i in ii:
                        matrix[4][i] = 1
        for n in range(len(sentences[index])):
            isNone = 1
            for m in range(5):
                if(matrix[m][n] == 1):
                    isNone = 0
            if(isNone == 1):
                matrix[0][n] = 1
        classes[index] = matrix

    return classes

In [9]:
def findClasses(index, classes):
    return classes[index]


In [10]:
def getMatrix(dependencies, vectors, classes, index):
    temp = []
    matrixNN = dependencies[index]
    for i in range(len(matrixNN)):
        for j in range(len(matrixNN[0])):
            if(matrixNN[i][j] == 1):
                temp.append((i, j))
    temp = np.array(temp)
    adj = sp.coo_matrix((np.ones(temp.shape[0]), (temp[:, 0], temp[:, 1])),
                        shape=(matrixNN.shape[0], matrixNN.shape[0]), dtype=np.float32)
    matrixND = vectors[index]
    matrixNE = classes[index]
    matrixNE = np.transpose(matrixNE)
    return matrixND, adj, matrixNE

In [11]:
def sample_mask(idx, l):
    mask = np.zeros(l)
    mask[idx] = 1
    return np.array(mask, dtype=np.bool)

#### Split the dataset into training dataset, validation dataset and test dataset.

In [12]:
def getSplit(y):
    idx_train = range((y.shape[0]) / 2)
    idx_val = range((y.shape[0])/2, y.shape[0])
    idx_test = range((y.shape[0])/2, y.shape[0])
    y_train = np.zeros(y.shape, dtype=np.int32)
    y_val = np.zeros(y.shape, dtype=np.int32)
    y_test = np.zeros(y.shape, dtype=np.int32)
    y_train[idx_train] = y[idx_train]
    y_val[idx_val] = y[idx_val]
    y_test[idx_test] = y[idx_test]
    train_mask = sample_mask(idx_train, y.shape[0])
    return y_train, y_val, y_test, idx_train, idx_val, idx_test, train_mask

#### Eample of implementing data processing

In [13]:
sentence_set = 'data/CoNLL_sentence_train.txt'
dependencies_set = "output/sentence_dependency.txt"
wordvector_set = "output/word_vec.txt"
wordclass_set = "output/word_class.txt"

In [14]:
sentences, indexes = getSentences(sentence_set)

In [15]:
for x in list(sentences)[0:3]:
    print ("key {}, value {} ".format(x,  sentences[x]))

key 1, value ['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb'] 
key 2, value ['Peter', 'Blackburn'] 
key 3, value ['BRUSSELS', '1996-08-22'] 


In [ ]:
dependencies = getDependencies(dependencies_set, sentences)
vectors = getVectors(wordvector_set, sentences)
classes = getClass(wordclass_set, sentences)
MND, temp, MNE = getMatrix(dependencies, vectors, classes, indexes[19])
y_train, y_val, y_test, idx_train, idx_val, idx_test, train_mask = getSplit(MNE)

## Implemented our own model using graph convolutional network.

In [16]:
from __future__ import print_function

from keras.layers import Input, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2

from layers.graph import GraphConvolution
from utils import *
from preprocess import *

import os
import time
import numpy as np

Using TensorFlow backend.


In [17]:
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
# Define parameters
DATASET = 'cora'
FILTER = 'localpool'  # 'chebyshev'
MAX_DEGREE = 2  # maximum polynomial degree
SYM_NORM = True  # symmetric (True) vs. left-only (False) normalization
NB_EPOCH = 30
PATIENCE = 10  # early stopping patience

In [20]:
# Get data
# start = time.time()
# data
sentence_set = "data/CoNLL_sentence_train_slim.txt"
dependencies_set = "output/sentence_train_dependency.txt"
wordvector_set = "output/CoNLL_word_vec_train.txt"
wordclass_set = "data/CoNLL_class_train_slim.txt"
sentences, indexes = getSentences(sentence_set)
# select sentences that length is greater than 20

In [ ]:
dependencies = getDependencies(dependencies_set, sentences)
vectors = getVectors(wordvector_set, sentences)
classes = getClass(wordclass_set, sentences)
sentences_needed = select(sentences, 20)
indexes_needed = sentences_needed.keys()

In [21]:
def buildModel(A, X, y):
    if FILTER == 'localpool':
        """ Local pooling filters (see 'renormalization trick' in Kipf & Welling, arXiv 2016) """
        # print('Using local pooling filters...')
        # A = np.asarray(A)
        A_ = preprocess_adj(A, SYM_NORM)
        support = 1
        graph = [X, A_]
        G = [Input(shape=(None, None), batch_shape=(None, None), sparse=True)]

    elif FILTER == 'chebyshev':
        """ Chebyshev polynomial basis filters (Defferard et al., NIPS 2016)  """
        # print('Using Chebyshev polynomial basis filters...')
        L = normalized_laplacian(A, SYM_NORM)
        L_scaled = rescale_laplacian(L)
        T_k = chebyshev_polynomial(L_scaled, MAX_DEGREE)
        support = MAX_DEGREE + 1
        graph = [X] + T_k
        G = [Input(shape=(None, None), batch_shape=(None, None), sparse=True) for _ in range(support)]

    else:
        raise Exception('Invalid filter type.')

    X_in = Input(shape=(X.shape[1],))

    # Define model architecture
    # NOTE: We pass arguments for graph convolutional layers as a list of tensors.
    # This is somewhat hacky, more elegant options would require rewriting the Layer base class.
    H = Dropout(0.85)(X_in)
    H = GraphConvolution(25, support, activation='relu', kernel_regularizer=l2(5e-4))([H] + G)
    H = Dropout(0.85)(H)
    Y = GraphConvolution(y.shape[1], support, activation='softmax')([H] + G)

    # Compile model
    model = Model(inputs=[X_in] + G, outputs=Y)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001))

    return model, graph

In [22]:
def getGraph(A, X, y):
    if FILTER == 'localpool':
        """ Local pooling filters (see 'renormalization trick' in Kipf & Welling, arXiv 2016) """
        # print('Using local pooling filters...')
        # A = np.asarray(A)
        A_ = preprocess_adj(A, SYM_NORM)
        support = 1
        graph = [X, A_]

    elif FILTER == 'chebyshev':
        """ Chebyshev polynomial basis filters (Defferard et al., NIPS 2016)  """
        # print('Using Chebyshev polynomial basis filters...')
        L = normalized_laplacian(A, SYM_NORM)
        L_scaled = rescale_laplacian(L)
        T_k = chebyshev_polynomial(L_scaled, MAX_DEGREE)
        support = MAX_DEGREE + 1
        graph = [X] + T_k

    else:
        raise Exception('Invalid filter type.')

    return graph

In [ ]:
# Helper variables for main training loop
wait = 0
preds = None
best_train_loss = 99999

X, MNN, y = getMatrix(dependencies, vectors, classes, '1')
X, MNN, y = reshapeMatrix(X, MNN, y, 25)
A = toTuple(MNN)
y_train, y_val, y_test, idx_train, idx_val, idx_test, train_mask = getSplit(y)
model, graph = buildModel(A, X, y)

confusion_matrix = zeros([5, 5], dtype=int32)

In [ ]:
# Fit
for epoch in range(1, NB_EPOCH+1):

    # Log wall-clock time
    t = time.time()
    train_loss_sum = 0
    train_acc_sum = 0
    test_loss_sum = 0
    test_acc_sum = 0

    train_loss_avg = 0
    train_acc_avg = 0
    test_loss_avg = 0
    test_acc_avg = 0

    train_count = 0
    test_count = 0
    process_bar = ShowProcess(len(indexes_needed))



    #for index in indexes_needed:
    # np.random.shuffle(indexes_needed)
    for i in range(len(indexes_needed)):
        X, MNN, y = getMatrix(dependencies, vectors, classes, indexes_needed[i])
        X, MNN, y = reshapeMatrix(X, MNN, y, 25)

        if not np.any(MNN) :
        	continue
        A = toTuple(MNN)
        y_train, y_val, y_test, idx_train, idx_val, idx_test, train_mask = getSplit(y)
        graph = getGraph(A, X, y)

        train_loss = list()
        train_acc = list()
        test_loss = list()
        test_acc = list()
        if(i <= len(indexes_needed) * 0.8):
            # Single training iteration (we mask nodes without labels for loss calculation)
            model.fit(graph, y_train, sample_weight=train_mask,
                  batch_size=A.shape[0], epochs=1, shuffle=False, verbose=0)

            # Predict on full dataset
            preds = model.predict(graph, batch_size=A.shape[0])

            for line in range(len(preds)):
                true_index = 0
                for lab in range(len(y_train[line])):
                    if (y_train[line][lab] == 0):
                        true_index += 1
                    else:
                        break
                max_index = 0
                for pos in range(len(preds[line])):
                    if preds[line][pos] > preds[line][max_index]:
                        max_index = pos
                if(true_index >= 5 or max_index >= 5):
                    continue
                confusion_matrix[true_index][max_index] += 1

            # Train / validation scores
            train_loss, train_acc = evaluate_preds(preds, [y_train], [idx_train])
            train_count = train_count + 1
            train_loss_sum = train_loss_sum + train_loss[0]
            train_acc_sum = train_acc_sum + train_acc[0]
        else:
            # Predict on full dataset
            preds = model.predict(graph, batch_size=A.shape[0])



            # Train / validation scores
            test_loss, test_acc = evaluate_preds(preds, [y_train], [idx_train])
            test_count = test_count + 1
            test_loss_sum = test_loss_sum + test_loss[0]
            test_acc_sum = test_acc_sum + test_acc[0]

        process_bar.show_process()

    train_loss_avg = train_loss_sum#/train_count
    train_acc_avg = train_acc_sum/train_count
    test_loss_avg = test_loss_sum#/test_count
    test_acc_avg = test_acc_sum/test_count

    print("Epoch: {:04d}".format(epoch),
          "train_loss= {:.4f}".format(train_loss_avg),
          "train_acc= {:.4f}".format(train_acc_avg),
          "val_loss= {:.4f}".format(test_loss_avg),
          "val_acc= {:.4f}".format(test_acc_avg),
          "time= {:.4f}".format(time.time() - t))

    # Early stopping
    if train_loss_avg < best_train_loss:
        best_train_loss = train_loss_avg
        wait = 0
    else:
        if wait >= PATIENCE:
            print('Epoch {}: early stopping'.format(epoch))
            break
        wait += 1


print("x")

## Test model

In [23]:
from preprocess import *

In [24]:
sentence_set = "data/sentence.txt"
dependencies_set = "output/sentence_dependency.txt"
wordvector_set = "output/word_vec.txt"
wordclass_set = "output/word_class.txt"

In [ ]:
sentences, indexes = getSentences(sentence_set)
dependencies = getDependencies(dependencies_set, sentences)
vectors = getVectors(wordvector_set, sentences)
classes = getClass(wordclass_set, sentences)
MND, MNN, MNE = getMatrix(dependencies, vectors, classes, indexes[0])

In [ ]:
for i in range(1, 10):
    key = indexes[i]
    MNDi, MNNi, MNEi = getMatrix(dependencies, vectors, classes, indexes[i])
    MND = vstack((MND, MNDi))
    MNE = vstack((MNE, MNEi))
    right_top = zeros((MNN.shape[0], MNNi.shape[1]), dtype=int8)
    left_bottom = zeros((MNNi.shape[0], MNN.shape[1]), dtype = int8)
    left = vstack((MNN, left_bottom))
    right = vstack((right_top, MNNi))
    MNN = hstack((left, right))
adj = toTuple(MNN)
y_train, y_val, y_test, idx_train, idx_val, idx_test, train_mask = getSplit(MNE)

#### Finally the program get 86.02% of accuracy on the testing set.

## Conclusion

In this paper, we discussed some mature method to do semantic role labeling. In the last part, we have shown a GCN model to do named entity extraction on CoNLL dataset. We believe that with the development of new technology such as deep learning and artificial neural network, some useful research area such as semantic role labeling and named entity extraction in natural language processing, will get a lot of help with it.

## Future scope

Although we got 86.02% of accuracy for our model,which is even higher than the state-of-art approach, we still want to think further about how to improve our model. One problem for our model is that all sentences used to train our model are from CoNLL 2003, which collected sentences from newspapers and magazines. We can try more different kinds of tagged datasets to train our model in order to increase its ability to adapt new datasets. In addition, We can also think that how the length and complexity of each sentence affect the accuracy of the model. Collecting longer and more complex sentences with correct tags might help us to train a more accurate model.

In [ ]:
"""
@article{kipf2016semi,
  title={Semi-Supervised Classification with Graph Convolutional Networks},
  author={Kipf, Thomas N and Welling, Max},
  journal={arXiv preprint arXiv:1609.02907},
  year={2016}
}
"""